In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, average_precision_score

X_train = pd.read_csv('../data/X_train.csv')
X_test = pd.read_csv('../data/X_test.csv')
y_train = pd.read_csv('../data/y_train.csv').values.ravel()
y_test = pd.read_csv('../data/y_test.csv').values.ravel()

In [ ]:

input_dim = X_train.shape[1]


model = Sequential([

    Dense(64, activation='relu', input_dim=input_dim),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])


model.compile(optimizer='adam',
              loss='binary_crossentropy', 
              metrics=['AUC', 'Precision', 'Recall'])

model.summary()

c:\Users\Wlady\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         1,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,097 (16.00 KB)

 Trainable params: 4,097 (16.00 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:

neg, pos = np.bincount(y_train)
class_weight = {0: 1, 1: neg / pos}

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=32,
                    validation_split=0.2,
                    class_weight=class_weight,
                    callbacks=[early_stopping],
                    verbose=1)

Epoch 1/100
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - AUC: 0.9242 - Precision: 0.0464 - Recall: 0.8264 - loss: 0.9418 - val_AUC: 0.9863 - val_Precision: 0.1220 - val_Recall: 0.9036 - val_loss: 0.0786
Epoch 2/100
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - AUC: 0.9743 - Precision: 0.0580 - Recall: 0.9132 - loss: 0.3487 - val_AUC: 0.9827 - val_Precision: 0.0993 - val_Recall: 0.8916 - val_loss: 0.0781
Epoch 3/100
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - AUC: 0.9835 - Precision: 0.0552 - Recall: 0.9068 - loss: 0.3037 - val_AUC: 0.9851 - val_Precision: 0.0516 - val_Recall: 0.9398 - val_loss: 0.1160
Epoch 4/100
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - AUC: 0.9808 - Precision: 0.0478 - Recall: 0.9132 - loss: 0.3161 - val_AUC: 0.9882 - val_Precision: 0.0920 - val_Recall: 0.9036 - val_loss: 0.0780
Epoch 5/100
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - AUC: 0.9901 - Precision: 0.0683 - Recall: 0.9293 - loss: 0.2429 - val_AUC: 0.9815 - val_Precision: 0.0597 - val_Recall: 0.9277 - v

In [ ]:

y_pred_proba = model.predict(X_test).ravel()

y_pred = (y_pred_proba > 0.5).astype(int)

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)
ap = average_precision_score(y_test, y_pred_proba)

print("\nResultados da Rede Neural:")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")
print(f"AP: {ap:.4f}")

1781/1781 ━━━━━━━━━━━━━━━━━━━━ 1s 577us/step

Resultados da Rede Neural:
Precision: 0.1250
Recall: 0.8980
F1-Score: 0.2195
ROC-AUC: 0.9761
AP: 0.7431
